# Creation of the KM3NeT datasets
The script produces pseudo datasets from KM3NeT IRFs. As production takes quite long, you can either reduce the time binning of the data generated below or download the results externally - see how in the data/km3net - folder.

To plot the results of this data set generation, see the `Plot_KM3NeT_datasets.ipynb` notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm

import astropy.units as u
from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from astropy.time import Time

from gammapy.data import Observation
from gammapy.datasets import MapDataset, Datasets
from gammapy.maps import WcsGeom, MapAxis, WcsNDMap
from gammapy.makers import MapDatasetMaker
from gammapy.irf import PSF3D, EnergyDispersion2D, Background2D, EffectiveAreaTable2D
from gammapy.modeling.models import BackgroundModel

from regions import CircleSkyRegion

import os
import sys
sys.path.append('../src')
from flux_utils import SourceModel
import plot_utils

from configure_analysis import AnalysisConfig
analysisconfig = AnalysisConfig()

%matplotlib inline
plot_utils.mpl_settings()

In [2]:
# The source can be set in the analysis_config.yml file
source_name = analysisconfig.get_source()
model = SourceModel(source_name)
print("Working on source", source_name, "at position", model.position)

Working on source VelaX at position <SkyCoord (ICRS): (ra, dec) in deg
    (128.287, -45.19)>


# Visibility

First, the zenith angles under which the source can be observed from the KM3NeT (ARCA) position needs to be calculated.

In the analysis, the time_step is set such that an hourly binning is used for generation of the datasets. 
Here, for easier execution, a daily binning is set (parameter `hours = 24` in `analysis_config.yml`). Change `analysis_config.yml` if it needs.

In [3]:
pos_arca = EarthLocation.from_geodetic(
    lat=analysisconfig.get_value("latitude", "km3net_datasets.detector_position"), 
    lon=analysisconfig.get_value("longitude", "km3net_datasets.detector_position"), 
    height=analysisconfig.get_value("height", "km3net_datasets.detector_position")
)

print ("ARCA position:", pos_arca)

time_step = analysisconfig.get_value("hours", "km3net_datasets")*3600 # in seconds
start = Time('2019-01-01T00:00:00', format='isot').unix
end = Time('2020-01-01T00:00:00', format='isot').unix
times = np.linspace(start, end-time_step, int(365*24*3600/time_step))
obstimes = Time(times, format='unix')

frames = AltAz(obstime=obstimes, location=pos_arca)
print ("Time steps", time_step, "with step duration", analysisconfig.get_value("hours", "km3net_datasets"))

ARCA position: (4943908.93847172, 1426985.99079506, 3750019.30219182) m
Time steps 28800 with step duration 8


In [4]:
# transform source position into local coordinates for each time step
local_coords = model.position.transform_to(frames)

# look at the minimum and maximum zenith angle (0 = above, 180 = below)
zen_angles = local_coords.zen.value

# zenith angle binning
cos_zen_bins = np.linspace(-1, 1, analysisconfig.get_value("zenithbinning", "km3net_datasets"))
cos_zen_binc = 0.5 * (cos_zen_bins[:-1] + cos_zen_bins[1:])
zen_bins = np.arccos(cos_zen_bins) * 180 / np.pi
zen_binc = np.arccos(cos_zen_binc) * 180 / np.pi

# compute visibility time for each zenith angle bin
vis_hist = np.histogram(np.cos(zen_angles*np.pi/180), bins=cos_zen_bins)
vis_times = vis_hist[0] / vis_hist[0].sum() * u.yr

In [5]:
# determine which zenith angle bins are going to be used
bin_mask = (zen_binc > 80) & (vis_hist[0] > 0)

# how many datasets will be used
n_datasets = bin_mask.sum()

# compute dataset indices for all time bins
dataset_idx = np.digitize(zen_angles, bins=zen_bins)
# for what it was done?!
dataset_idx -= dataset_idx.min()

# compute masks for each zenith angle bin
zen_masks = []
for i in range(n_datasets):
    z1 = zen_bins[1:][bin_mask][i]
    z2 = zen_bins[:-1][bin_mask][i]
    zen_masks.append((zen_angles > z1) & (zen_angles < z2))
    
# compute average zenith angle for each zenith angle bin
zen_mean = [zen_angles[m].mean() for m in zen_masks]
for i in zen_mean: print(i)

160.63116486887048
139.1090246529909
125.90772606852988
114.66403651518051
104.40592321688483
94.59478747222855
84.4333213533851


## Creation of Datasets

### Setting of the geometry

for each of the zenith angle bins a dataset will be created with the corresponding live time

In [6]:
# Set up geometries

# set the true and reconstructed energy axes
energy_axis = MapAxis.from_bounds(1e2, 1e6, nbin=16, unit='GeV', name='energy', interp='log')
energy_axis_true = MapAxis.from_bounds(1e2, 1e7, nbin=80, unit='GeV', name='energy_true', interp='log')

# also set the energy migration axis for the energy dispersion and the theta axis for the PSF
migra_axis = MapAxis.from_edges(np.logspace(-5, 2, 57), name='migra')
theta_axis = MapAxis.from_edges(np.linspace(0, 8, 101), name='theta', unit='deg')

# create the geometries
geom = WcsGeom.create(
    binsz  = 0.1*u.deg,
    width  = 30*u.deg,
    skydir = model.position,
    frame  = 'icrs',
    axes   = [energy_axis],
)
geom_true = WcsGeom.create(
    binsz  = 0.1*u.deg,
    width  = 30*u.deg,
    skydir = model.position,
    frame  = 'icrs',
    axes=[energy_axis_true],
)

IACT arrays have a pointing position, so we need to choose one here as well. Usually, the pointing position is used to compute an offset angle, which is then used to evaluate the IRFs. Below, we define some patches that make sure we evaluate the background model and the effective area at the correct zenith angle for each spatial pixel and time bin. The PSF and EDISP (which vary less strongly) are still evaluated based on the angle between the pointing we define here and the source position. Therefore, we simply rotate the source position along the declination axis.

In [7]:
livetime_pointings = vis_times[bin_mask] * 10  # 10 years

pointings = model.position.directional_offset_by(0*u.deg, np.array(zen_mean)*u.deg)

In [8]:
# transform all map coordinates into local coordinate system
map_coord = geom_true.to_image().get_coord()
sky_coord = map_coord.skycoord
map_coord_zeniths = [sky_coord.transform_to(frames[i]).zen.value for i in range(len(obstimes))]
map_coord_zeniths = np.array(map_coord_zeniths)

CPU times: user 37.4 s, sys: 2.04 s, total: 39.5 s
Wall time: 39.6 s


## Read KM3NeT IRF files

In [9]:
# read in the IRFs
aeff = EffectiveAreaTable2D.read(analysisconfig.get_file("km3net/irfs/aeff.fits"))
edisp = EnergyDispersion2D.read(analysisconfig.get_file("km3net/irfs/edisp.fits"))
psf = PSF3D.read(analysisconfig.get_file("km3net/irfs/psf.fits"))

# Also the neutrino / muon background
bkg_nu = Background2D.read(analysisconfig.get_file("km3net/irfs/bkg_nu.fits"))
bkg_mu = Background2D.read(analysisconfig.get_file("km3net/irfs/bkg_mu.fits"))

irfs = dict(aeff=aeff, psf=psf, edisp=edisp)

In [10]:
# Create observations
obs_list = [Observation.create(pointing=pointings[i], livetime=livetime_pointings[i], irfs=irfs)\
            for i in range(n_datasets)]

In [11]:
# Make the MapDatasets
empty = MapDataset.create(
    geom = geom, 
    energy_axis_true = energy_axis_true, 
    migra_axis = migra_axis, 
    rad_axis = theta_axis, 
    binsz_irf = 1
)

dataset_maker = MapDatasetMaker(selection=['exposure', 'psf', 'edisp'])

datasets = []
for i,obs in enumerate(obs_list):
    dataset = dataset_maker.run(empty.copy(name='nu{}'.format(i+1)), obs)
    datasets.append(dataset)

CPU times: user 11.9 s, sys: 2.08 s, total: 14 s
Wall time: 14.1 s


defining the pixel coordinates of the edges:
    
(top left, top center, top right, center left, src_pos, center right, bottom left, bottom center, bottom right)
of the considered field-of-view

In [12]:
edge_coord_pixel_pos = [(299,0), (299,150), (299,299),
                        (150,0), (150,150), (150,299),
                        (  0,0), (  0,150), (  0,299)]

We need to monkey-patch Background2D.evaluate, so that it evaluates the background at a certain zenith angle (instead of computing an offset angle). We simply pass the zenith angle as **fov_lon**.

Similarly, we need a patched method to evaluate the exposure correctly.

This method does the same as Gammapy's `make_map_exposure_true_energy`, except that it takes an offset angle (or, in our case, the zenith angle) instead of a pointing position.

In [13]:
def bkg_2d_eval_patched(self, fov_lon, fov_lat, energy_reco, method='linear', **kwargs):
    return self.data.evaluate(offset=fov_lon, energy=energy_reco, method='linear', **kwargs)

Background2D.evaluate = bkg_2d_eval_patched

# First, define a small helper function
def calc_exposure(offset, aeff, geom):
    energy = geom.get_axis_by_name('energy_true').center
    exposure = aeff.data.evaluate(offset=offset, energy_true=energy[:, np.newaxis, np.newaxis])
    return exposure

def make_map_exposure_true_energy_patched(offset, livetime, aeff, geom):
    exposure = calc_exposure(offset, aeff, geom)
    exposure = (exposure * livetime).to('m2 s')
    return WcsNDMap(geom, exposure.value.reshape(geom.data_shape), unit=exposure.unit)

## Compute background for all edge pixels and mean zenith angle
Below we compute the predicted background and the exposure for all edge pixels and all time bins

In [14]:
nu_background_edge_pixels = []
mu_background_edge_pixels = []
exposure_edge_pixels = []

d_omega = geom_true.to_image().solid_angle()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for pp in edge_coord_pixel_pos:
        zen_vals = map_coord_zeniths[:,pp[0],pp[1]]
    
        # compute neutrino background for every zenith angle value
        bkg_nu_de = bkg_nu.evaluate_integrate(
            fov_lon = zen_vals*u.deg,
            fov_lat = None, # does not matter
            energy_reco = geom.get_axis_by_name('energy').edges[:, np.newaxis]
        )
        bkg_nu_int = (bkg_nu_de * d_omega[pp[0],pp[1]]).to_value('s-1').sum(axis=0)
        nu_background_edge_pixels.append(bkg_nu_int)
    
        # compute muon background for every zenith angle value
        bkg_mu_de = bkg_mu.evaluate_integrate(
            fov_lon = zen_vals*u.deg,
            fov_lat = None, # does not matter
            energy_reco = geom.get_axis_by_name('energy').edges[:, np.newaxis]
        )
        bkg_mu_int = (bkg_mu_de * d_omega[pp[0],pp[1]]).to_value('s-1').sum(axis=0)
        mu_background_edge_pixels.append(bkg_mu_int)
    
        # compute exposure for every zenith angle value
        # (sum over all energies - just for this test)
        exp = (calc_exposure(zen_vals*u.deg, aeff, geom_true).sum(axis=(0,1)) * time_step * u.s).to_value('m2 s')
        exposure_edge_pixels.append(exp)

In [15]:
nu_background_edge_pixels_zen_mean = []
mu_background_edge_pixels_zen_mean = []
exposure_edge_pixels_zen_mean = []

for i in range(n_datasets):
    nu_background_edge_pixels_zen_mean.append([])
    mu_background_edge_pixels_zen_mean.append([])
    exposure_edge_pixels_zen_mean.append([])

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for pp in edge_coord_pixel_pos:
            zen_vals = map_coord_zeniths[:,pp[0],pp[1]][zen_masks[i]]
    
            # neutrino background
            bkg_nu_de = bkg_nu.evaluate_integrate(
                fov_lon = zen_vals.mean()*u.deg,
                fov_lat = None, # does not matter
                energy_reco = geom.get_axis_by_name('energy').edges[:, np.newaxis]
            )
            bkg_nu_int = (bkg_nu_de * d_omega[pp[0],pp[1]]).to_value('s-1').sum(axis=0)
            nu_background_edge_pixels_zen_mean[-1].append(bkg_nu_int[0])
    
            # muon background
            bkg_mu_de = bkg_mu.evaluate_integrate(
                fov_lon = zen_vals.mean()*u.deg,
                fov_lat = None, # does not matter
                energy_reco = geom.get_axis_by_name('energy').edges[:, np.newaxis]
            )
            bkg_mu_int = (bkg_mu_de * d_omega[pp[0],pp[1]]).to_value('s-1').sum(axis=0)
            mu_background_edge_pixels_zen_mean[-1].append(bkg_mu_int[0])
    
            # compute exposure for every zenith angle value
            # (sum over all energies - just for this test)
            exp = (calc_exposure(zen_vals.mean()*u.deg, aeff, geom_true).sum(axis=(0,1)) * time_step * u.s).to_value('m2 s')
            exposure_edge_pixels_zen_mean[-1].append(exp[0])

Given the plots above, it seems a good idea to compute the background rate and exposure separately for each pixel and time bin. Storing this in memory is not possible, so we immediately add up the background rates / exposure for each zenith angle bin.

!!! This takes O(day) for the original setup, or ~1h for a daily scan, depending on computing speed !!!

You can also download the data sets from our data server!

In [16]:
nu_background_maps = np.zeros((n_datasets, *geom.data_shape))
mu_background_maps = np.zeros((n_datasets, *geom.data_shape))
exposure_maps = np.zeros((n_datasets, *geom_true.data_shape))

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i in tqdm(range(len(obstimes)), "generation"):  
        # skip this time bin if the zenith angle of the source position is below 80 deg
        if dataset_idx[i] >= n_datasets:
            assert zen_angles[i] <= zen_bins[7]
            continue
    
        # zenith angle map for this time bin
        zen_vals = map_coord_zeniths[i]
    
        # compute neutrino background map for this time bin
        bkg_nu_de = bkg_nu.evaluate_integrate(
            fov_lon = zen_vals*u.deg,
            fov_lat = None, # does not matter
            energy_reco = geom.get_axis_by_name('energy').edges[:, np.newaxis, np.newaxis]
        )
        bkg_nu_de = (bkg_nu_de * d_omega).to_value('s-1')
    
        # add neutrino background rate for the correct dataset
        nu_background_maps[dataset_idx[i]] += bkg_nu_de
    
        # compute muon background map for this time bin
        bkg_mu_de = bkg_mu.evaluate_integrate(
            fov_lon = zen_vals*u.deg,
            fov_lat = None, # does not matter
            energy_reco = geom.get_axis_by_name('energy').edges[:, np.newaxis, np.newaxis]
        )
        bkg_mu_de = (bkg_mu_de * d_omega).to_value('s-1')
    
        # add muon background rate for the correct dataset
        mu_background_maps[dataset_idx[i]] += bkg_mu_de
    
        # compute exposure for this time bin
        exp = (calc_exposure(zen_vals*u.deg, aeff, geom_true) * time_step * u.s).to_value('m2 s')
    
        # add exposure for the correct dataset
        exposure_maps[dataset_idx[i]] += exp

generation: 100%|████████████████████████████████| 1095/1095 [14:26<00:00,  1.26it/s]

CPU times: user 12min 27s, sys: 1min 56s, total: 14min 24s
Wall time: 14min 26s


In [17]:
# Multiply exposure maps by 10, for 10 years of data
exposure_maps *= 10

In [18]:
# Replace the exposure attribute for each dataset
for i in range(n_datasets):
    exp_map = WcsNDMap(geom_true.copy(), data=exposure_maps[i], unit='m2 s')
    datasets[i].exposure = exp_map

divide the summed-up background rates for each dataset by the number of time bins that contribute to this dataset $\rightarrow$ obtain an averaged rate, 
then multiply with the total livetime for each dataset

In [19]:
for i in range(n_datasets):
    n_times = (dataset_idx == i).sum()
    nu_background_maps[i] /= n_times
    mu_background_maps[i] /= n_times

    nu_background_maps[i] *= livetime_pointings[i].to_value('s')
    mu_background_maps[i] *= livetime_pointings[i].to_value('s')

In [20]:
# Create the atmospheric neutrino background model
bkg_models = []
for i in range(n_datasets):
    # create map
    bkg_model_map = WcsNDMap(geom.copy(), data=nu_background_maps[i])

    # generate BackgroundModel instance
    bkg_model = BackgroundModel(bkg_model_map, name=datasets[i].name + '-bkg', datasets_names=[datasets[i].name])

    # store model
    bkg_models.append(bkg_model)

In [21]:
# Same for the atmospheric muon background
bkg_models_mu = []
for i in range(n_datasets):
    # Create map
    bkg_model_map = WcsNDMap(geom.copy(), data=mu_background_maps[i])

    # generate BackgroundModel instance
    bkg_model = BackgroundModel(bkg_model_map, name=datasets[i].name + 'bkg-mu', datasets_names=[datasets[i].name])

    # store model
    bkg_models_mu.append(bkg_model)

In [22]:
# First attach only neutrino background model to datasets, to compute npred
npred_bkg_nu = []
for i in range(n_datasets):
    datasets[i].models = bkg_models[i]
    npred_bkg_nu.append(datasets[i].npred())

In [23]:
# Now add the muon background (store npred for both the sum and the muon background only)
npred_bkg_mu = []
npred_bkg_sum = []
for i in range(n_datasets):
    datasets[i].background_model.stack(bkg_models_mu[i])
    bkg_sum = datasets[i].npred()
    bkg_mu = bkg_sum - npred_bkg_nu[i]
    npred_bkg_mu.append(bkg_mu)
    npred_bkg_sum.append(bkg_sum)

In [24]:
# Write the datasets to disk
if analysisconfig.get_value("write_KM3NeT_pseudodata", "io"):
    ext = analysisconfig.get_value("km3net_pseudodata_extension", "io")  # folder name extension when using different data sets
    ds_dir = analysisconfig.get_file("km3net/pseudodata/KM3NeT_"+source_name+ext)
    os.makedirs(ds_dir, exist_ok=True)
    Datasets(datasets).write(str(ds_dir) + "/KM3NeT_"+source_name+ext,
                             str(ds_dir) + "/KM3NeT_"+source_name+ext, overwrite=True)

In [25]:
# Also store npred maps for nu and mu background
if analysisconfig.get_value("write_KM3NeT_pseudodata", "io"):
    bkg_dir = analysisconfig.get_file(str(ds_dir)+"/npred_bkg_km3net")
    os.makedirs(bkg_dir, exist_ok=True)
    for i in range(n_datasets):
        nuname = str(bkg_dir) + "/{}_npred_nu_{:02d}.fits".format(source_name, i+1)
        muname = str(bkg_dir) + "/{}_npred_mu_{:02d}.fits".format(source_name, i+1)
        npred_bkg_nu[i].write(nuname, overwrite=True)
        npred_bkg_mu[i].write(muname, overwrite=True)